In [112]:
import torch
from transformers import BertTokenizer, BertModel, AutoTokenizer, AutoModelForTokenClassification

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
%matplotlib inline
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset, SequentialSampler
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import os
import copy
import ast
import cv2
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load pre-trained model tokenizer (vocabulary)
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-large-NER")



Downloading: 100%|██████████| 40.0/40.0 [00:00<00:00, 20.0kB/s]
Downloading: 100%|██████████| 1.42k/1.42k [00:00<00:00, 483kB/s]
Downloading: 100%|██████████| 208k/208k [00:00<00:00, 19.4MB/s]
Downloading: 100%|██████████| 112/112 [00:00<00:00, 56.0kB/s]
Downloading: 100%|██████████| 1.24G/1.24G [00:56<00:00, 23.8MB/s]


In [113]:
def verify_tokens(_tokenizer):
    tensor = _tokenizer.encode("How is life barack obama abcdde wuhan china 2020", return_tensors = "pt", add_special_tokens = True)
    for _tensor in tensor.view(-1):
        print(_tensor, _tokenizer.decode(_tensor))
    
verify_tokens(tokenizer)

tensor(101) [CLS]
tensor(1731) How
tensor(1110) is
tensor(1297) life
tensor(2927) bar
tensor(8741) ##ack
tensor(184) o
tensor(2822) ##ba
tensor(1918) ##ma
tensor(170) a
tensor(1830) ##b
tensor(1665) ##c
tensor(13976) ##dd
tensor(1162) ##e
tensor(192) w
tensor(23698) ##uh
tensor(1389) ##an
tensor(5144) chin
tensor(1161) ##a
tensor(12795) 2020
tensor(102) [SEP]


In [141]:
import pandas as pd
train = pd.read_parquet("../../Dataset/train.parquet")
valid = pd.read_parquet("../../Dataset/valid.parquet")

In [142]:
train.head(3)

,OCR,image,hero,villain,victim,other,_Hero,_Villain,_Victim,_Other,Target,All_Entities
0,Bernie Elizabeth? Be informed.Compare issues m...,covid_memes_18.png,[],[],[],"[bernie sanders, elizabeth warren]",,,,[BOTH] bernie sanders [SEP] elizabeth warren [...,[BOTH] bernie sanders [SEP] elizabeth warren [...,"[bernie sanders, elizabeth warren]"
1,Extending Brexit deadline October 31st order e...,covid_memes_19.png,[],[uk government],[],[],,[BVIL] uk government [EVIL],,,[BVIL] uk government [EVIL],[uk government]
2,kwai gkwa 0964 #nnevvy applause Thais Hong Kon...,covid_memes_252.png,[thais],[],[],[hong kong],[BH] thais [EH],,,[BOTH] hong kong [EOTH],[BH] thais [EH] [BOTH] hong kong [EOTH],"[thais, hong kong]"


In [146]:
#val = ast.literal_eval(train["BERT_Target_Entity"].values[0])
#len(val[0])
#train["BERT_Target_Entity"].values[0].strip('][').split(', ')


In [ ]:
for index in range(len(train)):
    image = train.loc[index, "image"]
    # Get image features
    image_feats, prediction = getImageFeatures(image)
    train.loc[index, ImageFeatures]

In [154]:
# Explode dataset based on category column and merge it
def getexplodedds(ds):
    data = ds.copy().reset_index()
    df = pd.DataFrame()
    for idx,col in enumerate(["hero","villain","victim","other"]):
        _df = data[["index","OCR","image",col]].explode(col)
        _df = _df[~_df[col].isna()]
        _df.columns = ["ID","OCR","image","Entity"]
        _df["Target"] = idx
        _df["TargetLabel"] = col
        df = pd.concat([df,_df], axis = 0, ignore_index = True)

    return df

train_df = getexplodedds(train)
valid_df = getexplodedds(valid)

In [157]:
train_df[train_df["ID"] == 0].head(3)

,index,OCR,Entity,Target,TargetLabel
3812,0,Bernie Elizabeth? Be informed.Compare issues m...,bernie sanders,3,other
3813,0,Bernie Elizabeth? Be informed.Compare issues m...,elizabeth warren,3,other


In [93]:
# resample skewed classes
def resample(ds, target, repeat_freq):
    data = ds.copy()
    _df = data.loc[data.TargetLabel == target].copy()
    for idx in range(repeat_freq):
        data = pd.concat([data, _df], axis = 0, ignore_index=True)
    return data

train_df = resample(train_df, "hero", 10)
train_df = resample(train_df, "victim", 6)
train_df = resample(train_df, "villain", 3)

train_df.TargetLabel.value_counts()

other      13702
villain     9708
victim      6370
hero        5225
Name: TargetLabel, dtype: int64

In [94]:
train_df = train_df.sample(frac=1).reset_index(drop = True)
train_df.head(3)

,OCR,Entity,Target,TargetLabel
0,CORONAVIRUS RACIST BACKLASH,racism,1,villain
1,WE SHALL OVER COMB ARE THESE TWO REALLY THE BE...,donald trump,1,villain
2,bydevumm THE WAY K.L RAHUL IS PERFORMING ANY G...,k.l. rahul,0,hero


In [158]:
# Tweet length distribution
import plotly.express as px
tweet_len = list(map(len, train_df.OCR.values))
px.histogram(tweet_len)

In [159]:
tweet_len = list(map(len, train_df.Entity.values))
px.histogram(tweet_len)

In [ ]:
# Extract Image Features
def getImageModel():
    
    checkpoint = torch.load("../../Models/img_class_ed.pt")
    class_names = checkpoint["class_names"]

    model_ft = models.resnet18(pretrained=True)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, len(class_names))
    model_ft.load_state_dict(checkpoint['model_state_dict'])

    model_ft.eval()
    model_ft = model_ft.to(device)
    return model_ft, class_names

def getImageFeatures(imageName):
    imageModel, class_names = getImageModel()

    transfomations = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((256, 256)),
        transforms.CenterCrop((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.4372, 0.4457, 0.4807],[0.3319, 0.3318, 0.3384])
    ])
    if imageName.lower().startsWith("covid"):
        root = "../../Dataset/covid/images"
    else:
        root = "../../Dataset/politics/images"
    img_name = os.path.join(root, imageName)
    image = cv2.imread(img_name)
    image = transfomations(image)

    outputs = imageModel(image)
    _, preds = torch.max(outputs, 1)

    imagePrediction = class_names.loc[class_names.Labels == preds[0].item(), "LabelName"].values[0]
    return image, imagePrediction


In [95]:
def getDataLoader(data, batch_size):
    # Tokenize all of the sentences and map the tokens to thier word IDs.
    input_ids_sent = []
    attention_masks_sent = []

    input_ids_ent = []
    attention_masks_ent = []

    # For every sentence...
    for id, sent, image, entity in zip(data.ID.values, data.OCR.values, data.image.values, data.Entity.values):
        
        # `encode_plus` will:
        #   (1) Tokenize the sentence.
        #   (2) Prepend the `[CLS]` token to the start.
        #   (3) Append the `[SEP]` token to the end.
        #   (4) Map tokens to their IDs.
        #   (5) Pad or truncate the sentence to `max_length`
        #   (6) Create attention masks for [PAD] tokens.
        encoded_dict = tokenizer.encode_plus(
                            sent,                      # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = 200,           # Pad & truncate all sentences.
                            pad_to_max_length = True,
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                    )

        encoded_dict_ent = tokenizer.encode_plus(
                            entity,                      # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = 20,           # Pad & truncate all sentences.
                            pad_to_max_length = True,
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                    )
        
        # Add the encoded sentence to the list.    
        input_ids_sent.append(encoded_dict['input_ids'])
        input_ids_ent.append(encoded_dict_ent['input_ids'])
        
        # And its attention mask (simply differentiates padding from non-padding).
        attention_masks_sent.append(encoded_dict['attention_mask'])
        attention_masks_ent.append(encoded_dict_ent['attention_mask'])

    # Convert the lists into tensors.
    input_ids_sent = torch.cat(input_ids_sent, dim=0)
    attention_masks_sent = torch.cat(attention_masks_sent, dim=0)

    input_ids_ent = torch.cat(input_ids_ent, dim=0)
    attention_masks_ent = torch.cat(attention_masks_ent, dim=0)
    labels = torch.tensor(data.Target.values)

    
    # Create the DataLoader.
    prediction_data = TensorDataset(id, input_ids_sent, attention_masks_sent, input_ids_ent, attention_masks_ent, labels)
    prediction_sampler = SequentialSampler(prediction_data)
    prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler
    , batch_size=batch_size)

    return prediction_dataloader


In [96]:
# Set the batch size.  
batch_size = 32  

dataloaders = {}
dataloaders["train"] = getDataLoader(train_df,batch_size)
dataloaders["val"] = getDataLoader(valid_df,batch_size)

dataset_sizes = {}
dataset_sizes["train"] = len(train_df)
dataset_sizes["val"] = len(valid_df)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Miniconda\envs\nlp\lib\site-packages\transformers\tokenization_utils_base.py:2263: FutureWarning:

The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).



In [108]:
class OCRClassification(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=(2,10), padding='same')
        self.maxpool = nn.MaxPool2d(kernel_size=(1,5))
        self.conv2 = nn.Conv2d(16, 32, (2,10),padding='same')
        self.conv3 = nn.Conv2d(32, 64, (2,10), padding="same")
        self.fc1 = nn.Linear(64 * 2 * 6, 100)
        self.fc2 = nn.Linear(100, 4)
        self.dropout = nn.Dropout(0.2)

    def forward(self, sent, img, entity):
        x = self.dropout(self.maxpool(F.relu(self.conv1(x))))
        x = self.dropout(self.maxpool(F.relu(self.conv2(x))))
        x = self.dropout(self.maxpool(F.relu(self.conv3(x))))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x




In [106]:
def train_model(model, bertmodel, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for b_input_ids_sent, b_input_mask_sent, b_input_ids_ent, b_input_mask_ent, labels in dataloaders[phase]:
                # Get output from bert model
                b_input_ids_sent, b_input_mask_sent, b_input_ids_ent, b_input_mask_ent= b_input_ids_sent.to(device), b_input_mask_sent.to(device), b_input_ids_ent.to(device), b_input_mask_ent.to(device)
                outputs_sent = bertmodel(b_input_ids_sent, token_type_ids=None, 
                                attention_mask=b_input_mask_sent)

                outputs_ent = bertmodel(b_input_ids_ent, token_type_ids=None, 
                                attention_mask=b_input_mask_ent)

                # Get hidden states from output
                hidden_states_sent = outputs_sent[2]
                hidden_states_ent = outputs_ent[2]

                # Stack all token embeddings from bert model
                token_embeddings_sent = torch.stack(hidden_states_sent, dim=0)
                token_embeddings_ent = torch.stack(hidden_states_ent, dim=0)

                # Take second last layer from hidden state and take a mean across tokens to get sentence level encoding
                # This will give an output of shape (batch_size * 768)
                sentence_embedding = torch.mean(token_embeddings_sent[-2,...], dim=1)
                entity_embedding = torch.mean(token_embeddings_ent[-2,...], dim=1)
                
                sentence_embedding = sentence_embedding.unsqueeze(dim = 1)
                entity_embedding = entity_embedding.unsqueeze(dim = 1)

                inputs = torch.cat([sentence_embedding, entity_embedding], dim=1).unsqueeze(dim=1).to(device)
                labels = labels.to(device)
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save({
                            'epoch': epoch,
                            'model_state_dict': best_model_wts,
                            'loss': loss,
                            }, "../../Models/ocr_class.pt")

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [109]:
model_ft = OCRClassification()
model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

#bertmodel = BertModel.from_pretrained('bert-base-uncased'
#                                       ,output_hidden_states = True, # Whether the model returns all hidden-states.
#                                  )

bertmodel = AutoModelForTokenClassification.from_pretrained("dslim/bert-large-NER", output_hidden_states=True)
for param in bertmodel.parameters():
    param.requires_grad = False

bertmodel = bertmodel.to(device)
bertmodel.eval()

model_ft = train_model(model_ft, bertmodel, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=20)                                 

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch 0/19
----------
train Loss: 1.3180 Acc: 0.3887
val Loss: 1.0771 Acc: 0.7680

Epoch 1/19
----------
train Loss: 1.3117 Acc: 0.3918
val Loss: 1.0585 Acc: 0.7680

Epoch 2/19
----------
train Loss: 1.3059 Acc: 0.4046
val Loss: 1.0428 Acc: 0.7627

Epoch 3/19
----------
train Loss: 1.2959 Acc: 0.4189
val Loss: 1.0626 Acc: 0.7129

Epoch 4/19
----------
train Loss: 1.2792 Acc: 0.4409
val Loss: 1.0520 Acc: 0.6965

Epoch 5/19
----------
train Loss: 1.2573 Acc: 0.4605
val Loss: 1.0638 Acc: 0.6684

Epoch 6/19
----------
train Loss: 1.2416 Acc: 0.4665
val Loss: 1.0816 Acc: 0.6578

Epoch 7/19
----------
train Loss: 1.2311 Acc: 0.4692
val Loss: 1.0867 Acc: 0.6525

Epoch 8/19
----------
train Loss: 1.2282 Acc: 0.4687
val Loss: 1.0839 Acc: 0.6530

Epoch 9/19
----------
train Loss: 1.2251 Acc: 0.4692
val Loss: 1.0848 Acc: 0.6491

Epoch 10/19
----------
train Loss: 1.2218 Acc: 0.4698
val Loss: 1.0814 Acc: 0.6525

Epoch 11/19
----------
train Loss: 1.2224 Acc: 0.4712
val Loss: 1.0800 Acc: 0.6506

Ep

In [57]:
#for batch in prediction_dataloader:
batch = next(iter(dataloaders["train"]))
# Add batch to GPU

# Unpack the inputs from our dataloader
b_input_ids_sent, b_input_mask_sent, b_input_ids_ent, b_input_mask_ent = batch

# Telling the model not to compute or store gradients, saving memory and 
# speeding up prediction
with torch.no_grad():
    # Forward pass, calculate logit predictions
    outputs_sent = model(b_input_ids_sent, token_type_ids=None, 
                    attention_mask=b_input_mask_sent)

    outputs_ent = model(b_input_ids_ent, token_type_ids=None, 
                    attention_mask=b_input_mask_ent)

    hidden_states_sent = outputs_sent[2]
    hidden_states_ent = outputs_ent[2]

# Concatenate the tensors for all layers. We use `stack` here to
# create a new dimension in the tensor.
token_embeddings_sent = torch.stack(hidden_states_sent, dim=0)
token_embeddings_ent = torch.stack(hidden_states_sent, dim=0)

#print(token_embeddings_sent.size())
sentence_embedding = torch.mean(token_embeddings_sent[-2,...], dim=1)
entity_embedding = torch.mean(token_embeddings_ent[-2,...], dim=1)
sentence_embedding.shape, entity_embedding.shape